<a href="https://colab.research.google.com/github/alanaquinoslv/curso-spark/blob/main/cursoSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok authtoken 2Sl4RhH8pY65GRGTXdNqzwxxkXC_4HnfVHCY7zottGi5h9i6U')
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://49b9-35-245-79-54.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://49b9-35-245-79-54.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
data = [('Zeca','35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']
df = spark.createDataFrame(data, colNames)
df

DataFrame[Nome: string, Idade: string]

In [ ]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [ ]:
df.toPandas()

,Nome,Idade
0,Zeca,35
1,Eva,29


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/empresas.zip', 'r').extractall('/content/drive/MyDrive/curso-spark')

In [ ]:
# criando dataframes
path = '/content/drive/MyDrive/curso-spark/empresas'
empresas = spark.read.csv(path, sep=';', inferSchema=True)

In [ ]:
empresas.count()

4585679

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/socios.zip', 'r').extractall('/content/drive/MyDrive/curso-spark')
zipfile.ZipFile('/content/drive/MyDrive/curso-spark/estabelecimentos.zip', 'r').extractall('/content/drive/MyDrive/curso-spark')

In [ ]:
path2 = '/content/drive/MyDrive/curso-spark/estabelecimentos'
estabelecimentos = spark.read.csv(path2, sep=';', inferSchema=True)

path3 = '/content/drive/MyDrive/curso-spark/socios'
socios = spark.read.csv(path3, sep=';', inferSchema=True)

In [ ]:
socios.count()
estabelecimentos.count()

4836219

In [ ]:
#manipulando os dados
empresas.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [ ]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
for index, colName in enumerate(empresasColNames):
  empresas = empresas.withColumnRenamed(f"_c{index}", colName)

empresas.columns

['cnpj_basico',
 'razao_social_nome_empresarial',
 'natureza_juridica',
 'qualificacao_do_responsavel',
 'capital_social_da_empresa',
 'porte_da_empresa',
 'ente_federativo_responsavel']

In [ ]:
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']
for index, colName in enumerate(sociosColNames):
  socios = socios.withColumnRenamed(f"_c{index}", colName)

socios.columns

['cnpj_basico',
 'identificador_de_socio',
 'nome_do_socio_ou_razao_social',
 'cnpj_ou_cpf_do_socio',
 'qualificacao_do_socio',
 'data_de_entrada_sociedade',
 'pais',
 'representante_legal',
 'nome_do_representante',
 'qualificacao_do_representante_legal',
 'faixa_etaria']

In [ ]:
socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [101]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
for index, colName22 in enumerate(estabsColNames):
  estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)

estabelecimentos.columns

['ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel',
 'ente_federativo_responsavel']